# Extrayendo datos desde las api públicas de mercado libre

Con el metodo get de la librería requests se consumiran las apis. La primera api a consumir, será el diccionario de categorías existentes a la fecha en mercado libre.

In [1]:
import requests 
from tqdm import tqdm
import pandas as pd

In [2]:

r =requests.get('https://api.mercadolibre.com/sites/MCO/categories') #api de las categorias consultada en https://developers.mercadolibre.com.ar/es_ar/categoriza-productos


Se guardan los resultados como un .json y se recorre este diccionario pintando las diferentes categorías: 

In [3]:

diccat = r.json()
for cat in diccat:
    print(cat)

{'id': 'MCO1747', 'name': 'Accesorios para Vehículos'}
{'id': 'MCO441917', 'name': 'Agro'}
{'id': 'MCO1403', 'name': 'Alimentos y Bebidas'}
{'id': 'MCO1071', 'name': 'Animales y Mascotas'}
{'id': 'MCO1367', 'name': 'Antigüedades y Colecciones'}
{'id': 'MCO1368', 'name': 'Arte, Papelería y Mercería'}
{'id': 'MCO1384', 'name': 'Bebés'}
{'id': 'MCO1246', 'name': 'Belleza y Cuidado Personal'}
{'id': 'MCO40433', 'name': 'Boletas para Espectáculos'}
{'id': 'MCO1039', 'name': 'Cámaras y Accesorios'}
{'id': 'MCO1743', 'name': 'Carros, Motos y Otros'}
{'id': 'MCO1051', 'name': 'Celulares y Teléfonos'}
{'id': 'MCO1648', 'name': 'Computación'}
{'id': 'MCO1144', 'name': 'Consolas y Videojuegos'}
{'id': 'MCO172890', 'name': 'Construcción'}
{'id': 'MCO1276', 'name': 'Deportes y Fitness'}
{'id': 'MCO5726', 'name': 'Electrodomésticos'}
{'id': 'MCO1000', 'name': 'Electrónica, Audio y Video'}
{'id': 'MCO175794', 'name': 'Herramientas'}
{'id': 'MCO1574', 'name': 'Hogar y Muebles'}
{'id': 'MCO1499', 'name

La api que contiene las características de cada categoría es: "https://api.mercadolibre.com/sites/MCO/search?category=id_cat&offset=numerodehoja", donde id_cat es el id de la categoría de interés y numerodehoja es el número de offset a consultar, un ejemplo es: 

In [3]:
e =requests.get('https://api.mercadolibre.com/sites/MCO/search?category=MCO1071&offset=2')


Ahora se concatena el fragmento de api "https://api.mercadolibre.com/sites/MCO/search?" con el id de cada una de las categorías guardadas en el diccionario diccat, recorriendo este diccionario se hace el llamado de la api de cada categoía (tomando las 50 primeras hojas) y los resultados se guardan en el diccionario d_product.

In [4]:
offset= [0,50,100,150,200,250,300,400,450,500,550,600,650,700,750]
d_product=[]
for cat in tqdm(diccat):
    for i in offset:
        
        api = f'https://api.mercadolibre.com/sites/MCO/search?'
        cat_id = cat['id']
        catml= f'category={cat_id}&offset={i}'
        r =requests.get(f'{api}{catml}')
        d_product.append(r.json())
        

100%|██████████| 32/32 [11:49<00:00, 22.17s/it]


Hasta aca hemos guardado en el diccionario d_product toda la información de los productos de cada categoría (los productos que aparecen en las primeras 50 hojas), ahora se hará el llamado de la información relacionada con el vendedor, teniendo en cuenta que la api que contiene la información de los vendedores es "https://api.mercadolibre.com/users/id_seller", donde id_seller sera reemplazado por el número de id del vendedor, el cual lo podemos encontrar en el diccionario d_product.

de d_product traemos el id del vendedor para poder relacionarlo en la api de los usuarios, posteriormente se hace el llamado de un solo usuario para que podamos visualizar la información disponible del vendedor:

In [6]:
id_seller=d_product[0]["results"][0]["seller"]["id"]
callsell=f'https://api.mercadolibre.com/users/{id_seller}'

In [8]:
s =requests.get(callsell)
s.json()

{'id': 147458773,
 'nickname': 'GMCOMUNICACIONESY ELECTRONICA',
 'registration_date': '2013-10-14T13:45:44.000-04:00',
 'country_id': 'CO',
 'address': {'city': 'Puente Aranda', 'state': 'CO-DC'},
 'user_type': 'normal',
 'tags': ['normal', 'eshop', 'mshops'],
 'logo': None,
 'points': 6633,
 'site_id': 'MCO',
 'permalink': 'http://perfil.mercadolibre.com.co/GMCOMUNICACIONESY+ELECTRONICA',
 'seller_reputation': {'level_id': '5_green',
  'power_seller_status': 'platinum',
  'transactions': {'canceled': 875,
   'completed': 19404,
   'period': 'historic',
   'ratings': {'negative': 0.03, 'neutral': 0.03, 'positive': 0.94},
   'total': 20279}},
 'buyer_reputation': {'tags': []},
 'status': {'site_status': 'active'}}

Conociendo que información podemos disponer del producto y del vendedor, se hará un recorrido con metodo for en cada categoría guardada en dic_product, y luego se recorrerá cada uno de los nombres y los valores del objeto results de las categorías, para finalmente guardar toda la información en un DataFrame:

In [9]:
object_names = ['id', 'site_id', 'title', 'price', 'sale_price',
             'currency_id', 'available_quantity', 'sold_quantity', 'buying_mode',
             'listing_type_id', 'stop_time', 'condition', 'permalink', 'thumbnail', 
             'thumbnail_id', 'accepts_mercadopago', 'original_price', 
             'category_id', 'official_store_id', 'domain_id', 'catalog_product_id',
              'use_thumbnail_id'] # nombres a traer del objeto results de cada una de las api de categorías.
dataset_productos = pd.DataFrame()
for i in tqdm(d_product):
    for result in i["results"]:
        try: #se declara try pues se intentó sin este y el for se detenia por el error de ejecución: "results",
            # por tanto se manejara este error de ejecución como una excepción
    
            
            dataset = {'object_'+name: result[name] for name in object_names}#guardamos en "dataset" los nombres (y sus valores) 
            dataset['General_category_id'] = i['filters'][0]['values'][0]['id']
            dataset['General_category_name'] = i['filters'][0]['values'][0]['name']


            dataset['free_ship'] = result['shipping']['free_shipping']# creamos nuevas columnas en dataset para guardar
            dataset['ship_mode'] = result['shipping']['mode']         # la información de ship que esta dentro de una jerarqía
            dataset['ship_logistic_type'] = result['shipping']['logistic_type']
            dataset['ship_storepickup'] = result['shipping']['store_pick_up']



            id_seller=result["seller"]["id"] # de cada objeto results de cada categoría traemos el id del vendedor

            callsell=f'https://api.mercadolibre.com/users/{id_seller}' 
            s =requests.get(callsell) # con cada id se hace el llamado de la api que contiene la información del vendedor
            sj=s.json()
            dataset["seller_level"] = sj["seller_reputation"] ["level_id"] # se crean nuevas columnas en dataset para guardar la información del vendedor
            dataset["seller_power_status"] = sj["seller_reputation"] ["power_seller_status"]
            dataset["sellertransactions_canceled"] = sj["seller_reputation"]["transactions"] ["canceled"]
            dataset["sellertransactions_completed"] = sj["seller_reputation"]["transactions"] ["completed"]
            dataset["sellertransactions_total"] = sj["seller_reputation"]["transactions"] ["total"]
            dataset["sellertransactions_period"] = sj["seller_reputation"]["transactions"] ["period"]
            dataset["sellerrating_negative"] = sj["seller_reputation"]["transactions"] ["ratings"]["negative"]
            dataset["sellerrating_neutral"] = sj["seller_reputation"]["transactions"] ["ratings"]["neutral"]
            dataset["sellerrating_positive"] = sj["seller_reputation"]["transactions"] ["ratings"]["positive"]
            item = pd.DataFrame(dataset, index=[0])
            dataset_productos= pd.concat([item, dataset_productos], axis=0) # se concatena la información de cada categoria
            
        except:
            print("error de ejecución")
            
            

100%|██████████| 480/480 [2:02:04<00:00, 15.26s/it]


In [10]:
dataset_productos.shape

(23601, 37)

Puesto que llamar toda la información tardó cerca de 2 horas, se guradará el dataset en un csv, para trabajar en los siguientes puntos sobre este archivo:

In [12]:
ruta = "./files/dataset_productos.csv"
dataset_productos.to_csv(ruta,index=False)